In [8]:
from langdetect import detect
import xml.etree.ElementTree as ET
import json
import requests
import re
import pandas as pd
from pprint import pprint


In [27]:
df = pd.read_csv('clean_songs.csv')

df

,Name,Artist,Album,Popularity,Lyrics,Nb contributors
0,10:35,Tiësto,10:35,88,23 Contributors10:35 Lyrics[Intro]\nAll I know...,23
1,2 Become 1,Spice Girls,Spice,68,27 Contributors2 Become 1 Lyrics[Verse 1: Spor...,27
2,21 Reasons (feat. Ella Henderson),Nathan Dawe,21 Reasons (feat. Ella Henderson),81,16 Contributors21 Reasons Lyrics[Verse 1]\nOne...,16
3,25 To Life,Eminem,Recovery,64,175 Contributors25 to Life Lyrics[Produced by ...,175
4,5 Leaf Clover,Luke Combs,Gettin' Old,75,8 Contributors5 Leaf Clover Lyrics[Verse 1]\nW...,8
...,...,...,...,...,...,...
482,Your Heart Or Mine,Jon Pardi,Mr. Saturday Night,76,3 ContributorsYour Heart Or Mine Lyrics[Verse ...,3
483,Your Song,Elton John,Elton John (Remastered Version),0,82 ContributorsYour Song Lyrics[Verse 1]\nIt's...,82
484,You're Gorgeous,Babybird,Ugly Beautiful,56,8 ContributorsYou’re Gorgeous Lyrics[Intro]\nY...,8
485,You're Makin' Me High,Toni Braxton,Secrets,62,23 ContributorsYou’re Makin’ Me High Lyrics[Ve...,23


In [3]:
genres_musicaux = [
    "rock",
    "pop",
    "hip-hop",
    "rap",
    "country",
    "blues",
    "jazz",
    "classical",
    "electronic",
    "dance",
    "reggae",
    "r&b",
    "folk",
    "metal",
    "punk",
    "alternative",
    "indie",
    "soul",
    "funk",
    "disco",
    "gospel",
    "electronic dance music (edm)",
    "techno",
    "house",
    "trance",
    "dubstep",
    "ska",
    "classical",
    "opera",
    "world",
    "k-pop",
    "latin",
    "raggae",
    "heavy metal",
    "singer-songwriter",
    "rap rock",
    "country pop",
    "jazz fusion",
    "electropop",
    "acoustic",
    "pop rock",
    "rap metal",
    "progressive rock",
    "classical crossover",
]

In [37]:
df['Lyrics'] = df['Lyrics'].str.replace(r'^.*\\', '', regex=True)
df

,Name,Artist,Album,Popularity,Lyrics,Nb contributors,duration,listener,playcount,date,lang
0,10:35,Tiësto,10:35,88,23 Contributors10:35 Lyrics[Intro]\nAll I know...,23,172000,225838,1632741,"08 Nov 2022, 17:18",en
1,2 Become 1,Spice Girls,Spice,68,27 Contributors2 Become 1 Lyrics[Verse 1: Spor...,27,241000,306356,1506036,"28 Jul 2009, 01:11",en
2,21 Reasons (feat. Ella Henderson),Nathan Dawe,21 Reasons (feat. Ella Henderson),81,16 Contributors21 Reasons Lyrics[Verse 1]\nOne...,16,155000,97395,549994,N/A,en
3,25 To Life,Eminem,Recovery,64,175 Contributors25 to Life Lyrics[Produced by ...,175,241000,375150,2130071,"21 Jan 2022, 17:36",en
4,5 Leaf Clover,Luke Combs,Gettin' Old,75,8 Contributors5 Leaf Clover Lyrics[Verse 1]\nW...,8,210000,23061,146242,N/A,en
...,...,...,...,...,...,...,...,...,...,...,...
482,Your Heart Or Mine,Jon Pardi,Mr. Saturday Night,76,3 ContributorsYour Heart Or Mine Lyrics[Verse ...,3,160000,21290,120884,N/A,en
483,Your Song,Elton John,Elton John (Remastered Version),0,82 ContributorsYour Song Lyrics[Verse 1]\nIt's...,82,241000,1032769,6050203,"06 Sep 2009, 19:16",en
484,You're Gorgeous,Babybird,Ugly Beautiful,56,8 ContributorsYou’re Gorgeous Lyrics[Intro]\nY...,8,223000,91450,401292,N/A,en
485,You're Makin' Me High,Toni Braxton,Secrets,62,23 ContributorsYou’re Makin’ Me High Lyrics[Ve...,23,246000,146654,539690,"02 Dec 2008, 00:58",en


In [33]:
def detect_lang(lyrics):
    langue_detectee = detect(str(lyrics))
    print(str(lyrics))
    return langue_detectee

df["lang"]=detect_lang(df["Lyrics"])

df

0      23 Contributors10:35 Lyrics[Intro]\nAll I know...
1      27 Contributors2 Become 1 Lyrics[Verse 1: Spor...
2      16 Contributors21 Reasons Lyrics[Verse 1]\nOne...
3      175 Contributors25 to Life Lyrics[Produced by ...
4      8 Contributors5 Leaf Clover Lyrics[Verse 1]\nW...
                             ...                        
482    3 ContributorsYour Heart Or Mine Lyrics[Verse ...
483    82 ContributorsYour Song Lyrics[Verse 1]\nIt's...
484    8 ContributorsYou’re Gorgeous Lyrics[Intro]\nY...
485    23 ContributorsYou’re Makin’ Me High Lyrics[Ve...
486    11 ContributorsTranslationsRomanizationEnglish...
Name: Lyrics, Length: 487, dtype: object


,Name,Artist,Album,Popularity,Lyrics,Nb contributors,duration,listener,playcount,date,lang
0,10:35,Tiësto,10:35,88,23 Contributors10:35 Lyrics[Intro]\nAll I know...,23,172000,225838,1632741,"08 Nov 2022, 17:18",en
1,2 Become 1,Spice Girls,Spice,68,27 Contributors2 Become 1 Lyrics[Verse 1: Spor...,27,241000,306356,1506036,"28 Jul 2009, 01:11",en
2,21 Reasons (feat. Ella Henderson),Nathan Dawe,21 Reasons (feat. Ella Henderson),81,16 Contributors21 Reasons Lyrics[Verse 1]\nOne...,16,155000,97395,549994,N/A,en
3,25 To Life,Eminem,Recovery,64,175 Contributors25 to Life Lyrics[Produced by ...,175,241000,375150,2130071,"21 Jan 2022, 17:36",en
4,5 Leaf Clover,Luke Combs,Gettin' Old,75,8 Contributors5 Leaf Clover Lyrics[Verse 1]\nW...,8,210000,23061,146242,N/A,en
...,...,...,...,...,...,...,...,...,...,...,...
482,Your Heart Or Mine,Jon Pardi,Mr. Saturday Night,76,3 ContributorsYour Heart Or Mine Lyrics[Verse ...,3,160000,21290,120884,N/A,en
483,Your Song,Elton John,Elton John (Remastered Version),0,82 ContributorsYour Song Lyrics[Verse 1]\nIt's...,82,241000,1032769,6050203,"06 Sep 2009, 19:16",en
484,You're Gorgeous,Babybird,Ugly Beautiful,56,8 ContributorsYou’re Gorgeous Lyrics[Intro]\nY...,8,223000,91450,401292,N/A,en
485,You're Makin' Me High,Toni Braxton,Secrets,62,23 ContributorsYou’re Makin’ Me High Lyrics[Ve...,23,246000,146654,539690,"02 Dec 2008, 00:58",en


In [10]:
def get_lyrics(artist, song_name):
    url = f"http://api.chartlyrics.com/apiv1.asmx/SearchLyricDirect?artist={artist}&song={song_name}"

    response = requests.get(url)
    response_text = response.text

    if response.status_code == 200:
        lyric_match = re.search(r'<Lyric>(.*?)</Lyric>', response_text, re.DOTALL)

        if lyric_match:
            lyric_content = lyric_match.group(1)
            return(lyric_content)
        else:
            print("Balise <Lyric> introuvable.")

#print(get_lyrics("lady gaga","girl"))

In [28]:
import requests
import pandas as pd

def song_infos(row):
    artist_name = row["Artist"]
    song_name = row["Name"]
    api_key = 'fbd866cc9dd08da54a1e4ed4f6ff7896'

    url = f"http://ws.audioscrobbler.com/2.0/?method=track.getinfo&api_key={api_key}&artist={artist_name}&track={song_name}&format=json"
    response = requests.get(url)
    data = response.json()

    # Vérifiez si la clé 'track' est dans la réponse (pour gérer les erreurs)
    if 'track' in data:
        track_data = data['track']
        duration = int(track_data['duration'])
        listener = track_data['listeners']
        playcount = track_data['playcount']

        # Vérifiez si 'wiki' est présent dans les données (pour éviter les erreurs)
        if 'wiki' in track_data:
            published = track_data['wiki'].get('published', 'N/A')
        else:
            published = 'N/A'

        # Retourne un Series pandas avec les informations
        return pd.Series({'duration': duration, 'listener': listener, 'playcount': playcount, 'date': published})
    else:
        return pd.Series({'duration': 0, 'listener': 0, 'playcount': 0, 'date': 'N/A'})


new_info_df = df.apply(song_infos, axis=1)

# Concaténez le DataFrame d'origine avec le DataFrame des nouvelles informations
df = pd.concat([df, new_info_df], axis=1)

# Affichez le DataFrame mis à jour
print(df)


                                  Name        Artist  \
0                                10:35        Tiësto   
1                           2 Become 1   Spice Girls   
2    21 Reasons (feat. Ella Henderson)   Nathan Dawe   
3                           25 To Life        Eminem   
4                        5 Leaf Clover    Luke Combs   
..                                 ...           ...   
482                 Your Heart Or Mine     Jon Pardi   
483                          Your Song    Elton John   
484                    You're Gorgeous      Babybird   
485              You're Makin' Me High  Toni Braxton   
486                               アイドル       YOASOBI   

                                 Album  Popularity  \
0                                10:35          88   
1                                Spice          68   
2    21 Reasons (feat. Ella Henderson)          81   
3                             Recovery          64   
4                          Gettin' Old          75   
.. 

In [29]:
df

,Name,Artist,Album,Popularity,Lyrics,Nb contributors,duration,listener,playcount,date
0,10:35,Tiësto,10:35,88,23 Contributors10:35 Lyrics[Intro]\nAll I know...,23,172000,225838,1632741,"08 Nov 2022, 17:18"
1,2 Become 1,Spice Girls,Spice,68,27 Contributors2 Become 1 Lyrics[Verse 1: Spor...,27,241000,306356,1506036,"28 Jul 2009, 01:11"
2,21 Reasons (feat. Ella Henderson),Nathan Dawe,21 Reasons (feat. Ella Henderson),81,16 Contributors21 Reasons Lyrics[Verse 1]\nOne...,16,155000,97395,549994,N/A
3,25 To Life,Eminem,Recovery,64,175 Contributors25 to Life Lyrics[Produced by ...,175,241000,375150,2130071,"21 Jan 2022, 17:36"
4,5 Leaf Clover,Luke Combs,Gettin' Old,75,8 Contributors5 Leaf Clover Lyrics[Verse 1]\nW...,8,210000,23061,146242,N/A
...,...,...,...,...,...,...,...,...,...,...
482,Your Heart Or Mine,Jon Pardi,Mr. Saturday Night,76,3 ContributorsYour Heart Or Mine Lyrics[Verse ...,3,160000,21290,120884,N/A
483,Your Song,Elton John,Elton John (Remastered Version),0,82 ContributorsYour Song Lyrics[Verse 1]\nIt's...,82,241000,1032769,6050203,"06 Sep 2009, 19:16"
484,You're Gorgeous,Babybird,Ugly Beautiful,56,8 ContributorsYou’re Gorgeous Lyrics[Intro]\nY...,8,223000,91450,401292,N/A
485,You're Makin' Me High,Toni Braxton,Secrets,62,23 ContributorsYou’re Makin’ Me High Lyrics[Ve...,23,246000,146654,539690,"02 Dec 2008, 00:58"


In [11]:

import requests

# Remplacez "VOTRE_CLE_API" par votre clé d'API Last.fm
api_key = 'fbd866cc9dd08da54a1e4ed4f6ff7896'

# Remplacez "artiste" et "titre" par le nom de l'artiste et le titre de la chanson que vous souhaitez rechercher
artiste = "indochine"
titre = "l'aventurier"
tags_artist = {}
tags_song = {}
# Effectuez une requête pour obtenir des informations sur l'artiste
artist_info_url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={artiste}&api_key={api_key}&format=json"
response_artist = requests.get(artist_info_url)

# Effectuez une requête pour obtenir des informations sur la chanson
track_info_url = f"http://ws.audioscrobbler.com/2.0/?method=track.getInfo&artist={artiste}&track={titre}&api_key={api_key}&format=json"
response_track = requests.get(track_info_url)

# Vérifiez si les requêtes ont réussi (code de statut 200)
if response_artist.status_code == 200 and response_track.status_code == 200:
    artist_data = response_artist.json()
    track_data = response_track.json()

    # Extrait le genre musical de l'artiste
    artist_genre = artist_data.get("artist", {}).get("tags", {}).get("tag", [])[0].get("name", "Genre non trouvé")

    # Extrait le genre musical de la chanson (s'il est disponible)
    track_genre = track_data.get("track", {}).get("toptags", {}).get("tag", [])[0].get("name", "Genre non trouvé")

    track_tags = track_data.get("track", {}).get("toptags", {}).get("tag", [])
    artist_tags = track_data.get("artist", {}).get("tags", {}).get("tag", [])


    tags_song[f"{artiste} - {titre}"] = [tag["name"] for tag in track_tags if tag["name"] in genres_musicaux]
    tags_artist[f"{artiste}"] = [tag["name"] for tag in artist_tags if tag["name"] in genres_musicaux]
    print(tags_song)
    print(tags_artist)
else:
    print("Erreur lors de la récupération des informations.")


{'artist': {'bio': {'content': 'Indochine (French for "Indochina") was formed '
                               'in May 1981 in Paris, France by Nicola Sirkis '
                               'and Dominique Nicolas.  It took them a few '
                               'months to write their first songs. They soon '
                               'took on Dimitri Bodianski, the cousin of one '
                               'of Nicola’s friends. Very soon, Stephane '
                               "Sirkis, Nicola's brother got involved in the "
                               'band and composed most of their hits in which '
                               'Nicola always took the leading vocals.\n'
                               'They gave their first concert at "Le Rose '
                               'bonbon", a café in Paris on 29 September 1981. '
                               'This brief performance on stage earned them '
                               'their first contract with a re